In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline


from glob import glob
import os 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE

import torch
from torch import nn, optim
import timm

import random

#from PIL import image

import albumentations as A
from albumentations.pytorch import ToTensorV2

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm 

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
train = pd.read_csv('mnist/train.csv')
test = pd.read_csv('mnist/test.csv')
submission = pd.read_csv('mnist/sample_submission.csv')

In [4]:
train

,index,label,px1,px2,px3,px4,px5,px6,px7,px8,...,px775,px776,px777,px778,px779,px780,px781,px782,px783,px784
0,0,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,59995,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,59996,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,59997,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,59998,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
train_df = train.copy()

In [15]:
train_df.drop(['index', 'label'], axis=1, inplace=True)

In [49]:
try_0 = train_df.loc[0].to_numpy()

In [57]:
try_0 = np.reshape(try_0, (28,28,))

In [31]:
!pip install pillow

In [34]:
from PIL import Image

In [54]:
try_0 = try_0/255

In [59]:
try_0.shape

(28, 28)

In [55]:
Image.fromarray(try_0).show()

TypeError: Cannot handle this data type: (1, 1, 1), <f8

In [38]:
try_0.shape

(28, 28)

# seed, loss func

In [62]:
def seed_everything(seed) :
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    

def NMAE(true, pred) : 
    mae = np.mean(np.abs(true-pred))
    score = mae/np.mean(np.abs(true))
    return score

SEED = 42
seed_everything(SEED)

# Global Variable

In [ ]:
IMAGE_SIZE = 128
TEST_SIZE = 0.2
N_EPOCH = 3
LR = 1e-3
BATCH_SIZE = 32
MODEL_NAME = 'densenet121'


# Dataset

In [64]:
class train_dataset(Dataset) :
    def __init__(self, train_df, transform=None ) :
        super(train_dataset).__init__()
        self.train_df = train_df
        self.transform = transform 
        
    def __len__(self) : 
        return len(self.train_df)
    
    def __getitem__(self, idx) : 
        img = np.array(Image.open(self.train_df['image_path'][idx]))
        
        if self.transform : 
            img = self.transform(image=img)['image']
        
        weight = self.train_df['target'][idx]
        weight = torch.tensor(weight, dtype=torch.float32)
        
        return img, weight

In [65]:
class test_dataset(Dataset) : 
    def __init__(self, test_imgs, transform =None) :
        super(test_dataset).__init__() 
        self.test_imgs = test_imgs
        self.transform = transform
        
    def __len__() :
        return len(self.test_imgs)
    
    def __getitem__(self, idx) :  
        img = np.array(Image.open(self.test_imgs[idx]))
        
        if self.transform : 
            img = self.transform(image=img)['image']
            
        return img
        

# Dataloader, transform

In [63]:
train_loader = DataLoader(train_dataset(), batch_size = BATCH_SIZE, shuffle=True)
val_loadr = DataLoader(train_dataset(), batch_size = BATCH_SIZE, shuffle=False)

img_transform = A.Compose([
    A.Resize(IMAGE_SIZE, IMAGE_SIZE),
    A.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
    ToTensorV2()
])

NameError: name 'train_dataset' is not defined

# Model

In [66]:
# pretrained 
class ConvNet(nn.Module) : 
    def __init__(self, model_name, pretrained=True) : 
        super(ConvNet, self).__init__()
        self.conv = timm.create_model(model_name, pretrained=pretrained) # 전이학습 모델 불러오고, 
        self.fc1 = nn.Linear(1000, 256)  # fc 
        self.relu = nn.ReLU() # activation
        self.fc2 = nn.Linear(256,1) # fc  to 1 
        
        
    def forward(self, inputs) : 
        X = self.conv(inputs)
        X = self.fc1(X)
        X = self.relu(X)
        outputs = self.fc2(X)
        return outputs

In [67]:
# 직접 model 구성

class ConvNet(nn.Module) : 
    def __init__(self) : 
        super(ConvNet, self).__init__()
        self.layer1 = torch.nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer4 = torch.nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.regressor = nn.Linear(3136,1)
        
        
    def forward(self, x) : 
        # (batch, 3, image_size, image_size)-> batch, 64, resize, resize)
        # 128*128 -> 7*7
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        # 64*7*7 = 3136 
        x = torch.flatten(x, start_dim=1)
        out = self.regressor(x)
        return out

In [72]:
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss().to(device)  # Or L1loss()
optimizer = torch.optim.Adam(model.parameters(), lr = LR)

# Train/Validation

In [71]:
TEST_SIZE = 0.2
N_EPOCH = 30 
LR = 1e-3
BATCH_SIZE = 32
MODEL_NAME = 'resnet18'
IMAGE_SIZE = 256

**Train**

In [73]:
# dataframe 학습/평가로 나누고
train_data, val_data = train_test_split(df, test_size = TEST_SIZE, random_state = SEED)
train_data = train_data.reset_index(drop = True)
val_data = val_data.reset_index(drop = True)

# 각 데이터셋을 data loader로 읽어옴
train_loader = DataLoader(train_dataset(train_data), shuffle=True, batch_size = BATCH_SIZE)
val_loader = DataLoader(train_dataset(val_data), shuffle=True, batch_size = BATCH_SIZE)


for epoch in range(N_EPOCH) : 
    avg_cosv = 0
    
    for data, target in train_loader : 
        data = data.to(device)
        target = target.to(device)
        # target = target[..., None].to(device)
        optimizer.zero_grad()
        hypothesis = model(data) # 모델을 forward pass로 결과 저장 
        cost = criterion(hypothesis, target) # output과 target loss계산
        cost.backward() # backward해서 gradient계산
        optimizer.step() # 모델 파라미터 갱신 
        avg_cost += cost / len(train_loader)  # 평균 loss 계산 
    print(f'[Epoch: {epoch+1:>4}] cost = {avg_cost:>.9}')
    
    model.eval()
    with torch.no_grad() : # grad 해제 
        correct = 0
        total = 0

        for data, target in val_loader : 
            data = data.to(device)
            target = target.to(device) 
            # target = target[..., None].to(device)
            out = model(data)

            preds = torch.max(out.data, 1)[1]
            total += len(target) 
            correct += (preds==target).sum().item()

            print('TEST ACCURACY :  ', 100.*correct/total, '%')

NameError: name 'train_loader' is not defined

**Validation**

In [74]:
model.eval()
with torch.no_grad() : # grad 해제 
    correct = 0
    total = 0
    
    for data, target in val_loader : 
        data = data.to(device)
        target = target.to(device) 
        # target = target[..., None].to(device)
        out = model(data)
        
        preds = torch.max(out.data, 1)[1]
        total += len(target) 
        correct += (preds==target).sum().item()
        
        print('TEST ACCURACY :  ', 100.*correct/total, '%')

NameError: name 'val_loader' is not defined

# Train/validation def

In [ ]:
def train(model, optimizer, train_loader, vali_loader, scheduler, device):
    model.to(device)

    # Loss Function
    criterion = nn.L1Loss().to(device)
    best_mae = 9999
    
    for epoch in range(EPOCH):
        model.train()
        train_loss = []
        for img, label in tqdm(iter(train_loader)):
            img, label = img.float().to(device), label.float().to(device)
            
            optimizer.zero_grad()

            # Data -> Model -> Output
            logit = model(img)
            # Calc loss
            loss = criterion(logit.squeeze(1), label)

            # backpropagation
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())
            
        if scheduler is not None:
            scheduler.step()
            
        # Evaluation Validation set
        vali_mae = validation(model, vali_loader, criterion, device)
        
        print(f'Epoch [{epoch+1}] Train MAE : [{np.mean(train_loss):.5f}] Validation MAE : [{vali_mae:.5f}]\n')
        
        # Model Saved
        if best_mae > vali_mae:
            best_mae = vali_mae
            torch.save(model.state_dict(), './best_model.pth')
            print('Model Saved.')

In [ ]:
def validation(model, vali_loader, criterion, device):
    model.eval() # Evaluation
    vali_loss = []
    with torch.no_grad():
        for img, label in tqdm(iter(vali_loader)):
            img, label = img.float().to(device), label.float().to(device)

            logit = model(img)
            loss = criterion(logit.squeeze(1), label)
            
            vali_loss.append(loss.item())

    vali_mae_loss = np.mean(vali_loss)
    return vali_mae_loss

In [ ]:
CNNmodel = CNNRegressor().to(device)

optimizer = torch.optim.SGD(params = CNNmodel.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = None

train(CNNmodel, optimizer, train_loader, vali_loader, scheduler, device)

# Inference

In [ ]:
def predict(model, test_loader, device):
    model.eval()
    model_pred = []
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.float().to(device)

            pred_logit = model(img)
            pred_logit = pred_logit.squeeze(1).detach().cpu()

            model_pred.extend(pred_logit.tolist())
    return model_pred

In [ ]:
test_dataset = CustomDataset(processed_test_imgs, None, train_mode=False, transforms=test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# Validation Score가 가장 뛰어난 모델을 불러옵니다.
checkpoint = torch.load('./best_model.pth')
CNNmodel = CNNRegressor().to(device)
CNNmodel.load_state_dict(checkpoint)

# Inference
preds = predict(CNNmodel, test_loader, device)

In [ ]:
submission = pd.read_csv('./open/sample_submission.csv')
submission['leaf_weight'] = preds
submission.to_csv('./CNNsubmit.csv', index=False)